# HDT4 PREDICCIÓN

In [34]:
import nltk
from nltk import lm
from nltk.corpus import stopwords
from nltk.book import FreqDist
from nltk.util import ngrams
import matplotlib.pyplot as plt

In [3]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [56]:
tweets = []
with open('./clean_data/en_US.twitter.txt', 'r') as f:
    tweets = f.read()

with open('./clean_data/twitter.txt', 'w') as f:
    f.write(tweets)
    
with open('./clean_data/twitter.txt', 'r') as f:
    tweets = f.read()

### N Gramas

In [12]:
tweets_1_gram = list(ngrams(tweets, 1))
tweets_2_gram = list(ngrams(tweets, 2))
tweets_3_gram = list(ngrams(tweets, 3))

In [17]:
tweets_1_gram[:100]

NameError: name 'tweets_1_gram' is not defined

In [46]:
PERCENT_TO_TRAIN = 0.10
tweets_data = tweets[:round(len(tweets) * PERCENT_TO_TRAIN)]
len(tweets_data)

16445640

In [57]:
from nltk import word_tokenize, sent_tokenize
tokenized_text = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(tweets)]

In [48]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(3, tokenized_text)

In [49]:
from nltk.lm import MLE
mle = MLE(3)

In [50]:
mle.fit(train, vocab)
print(mle.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 110385 items>


In [51]:
mle.score('you', 'how are'.split())

0.754950495049505

In [52]:
mle.generate(3, 'how are'.split())

['you', 'interested', 'in']